In [ ]:
using Revise
using BenchmarkTools

using Kwat

In [ ]:
tr = joinpath("..", "tree", "")

;

In [ ]:
em = "1.empty.md"

he_ = [li for li in readlines(em) if li != ""]

;

In [ ]:
using Kwat.path: clean

function is_good_structure(na::String)::Bool
    
    if na != clean(na)
        
        error(na)
        
    end
    
    return occursin(r"^[0-9]+\.[_0-9a-z]+(\.md$|$)", na)
    
end

In [ ]:
function split_for_sorting(na::String)::Vector{Union{String,Float64}}

    pa_ = Vector{Union{String,Float64}}()

    for sp in split(na, ".")

        sp = string(sp)

        pu = tryparse(Float64, sp)

        if pu === nothing

            pu = sp

        end

        push!(pa_, pu)

    end

    return pa_

end

In [ ]:
for (ro, di_, fi_) in walkdir(tr; topdown = false)

    fi_ = [fi for fi in fi_ if !startswith(fi, ".")]

    for fi in fi_

        pa = joinpath(ro, fi)

        li_ = readlines(pa)

        if length(li_) == 0

            println("Is empty (populating): ", pa)

            cp(em, pa; force = true)

        elseif !all(he in li_ for he in he_)

            println("Is missing at least 1 required header: ", pa)

        elseif !all(0 .< diff([findfirst(li_ .== he) for he in he_]))

            println("Has incorrectly ordered headers: ", pa)

        end

    end

    go_ = Vector{String}()

    ba_ = Vector{String}()

    for na in [di_; fi_]

        if is_good_structure(na)

            push!(go_, na)

        else

            push!(ba_, na)

        end

    end

    sort!(go_; by = split_for_sorting)

    for (id, na) in enumerate([go_; ba_])

        pa1 = joinpath(ro, na)

        if is_good_structure(na)

            pa2 = joinpath(ro, string(id, ".", split(na, "."; limit = 2)[2]))

        else

            pa2 = joinpath(ro, string(id, ".", na))

        end

        if pa1 != pa2

            println(splitdir(pa1)[2], " ==> ", splitdir(pa2)[2])

            mv(pa1, pa2)

        end

    end

end